# Convert Calcium Imaging data from .npz to NWB file
More details on [NWB Calcium imaging data](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html#calcium-imaging-data).

**0.** We start importing the relevant modules to manipulate NWB file groups and datasets

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile, NWBHDF5IO, ProcessingModule
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, DfOverF, MotionCorrection
from pynwb.device import Device
from pynwb.base import TimeSeries
from pynwb.behavior import SpatialSeries, Position
from pynwb.image import ImageSeries
from ndx_grayscalevolume import GrayscaleVolume

import scipy.io
import numpy as np
import h5py
import os
import matplotlib.pyplot as plt

**1.** Importing the data in `.npz` files

In [ ]:
files_path = '/Users/bendichter/Desktop/Axel Lab/data/2019_07_01_fly2'

fname1 = '2019_07_01_Nsyb_NLS6s_walk_fly2.npz'
fpath1 = os.path.join(files_path, fname1)
file1 = np.load(fpath1)
print('First file:')
print('Groups:', file1.files)
print('Dims (height,width,depth):', file1['dims'])
print('Time shape: ', file1['time'].shape)
print('trialFlag shape: ', file1['trialFlag'].shape)
print('dFF shape: ', file1['dFF'].shape)
print('Ball shape: ', file1['ball'].shape)
print('dlc shape: ', file1['dlc'].shape)

fname2 = '2019_07_01_Nsyb_NLS6s_walk_fly2_A.npz'
fpath2 = os.path.join(files_path, fname2)
file2 = np.load(fpath2)
print('     ')
print('Second file - Sparse Matrix:')
print('Groups:', file2.files)
print('Indices: ', file2['indices'], '  | Shape: ',file2['indices'].shape)
print('Indptr: ', file2['indptr'], '  | Shape: ',file2['indptr'].shape)
print('Format: ', file2['format'])
print('Shape: ', file2['shape'])
print('Data: ', file2['data'], '  | Shape: ',file2['data'].shape)

fname3 = '2019_07_01_Nsyb_NLS6s_walk_fly2_ref_im.npz'
fpath3 = os.path.join(files_path, fname3)
file3 = np.load(fpath3)
print('   ')
print('Third file:')
print('Groups:', file3.files)
print('Im shape:', file3['im'].shape)

**2.** Create a new NWB file instance, OpticalChannel, ImagingPlane and ProcessingModule.

In [ ]:
#Create new NWB file
nwb = NWBFile(session_description='my CaIm recording', 
              identifier='EXAMPLE_ID', 
              session_start_time=datetime.now(tzlocal()),
              experimenter='Evan Schaffer',
              lab='Axel lab',
              institution='Columbia University',
              experiment_description='EXPERIMENT_DESCRIPTION',
              session_id='IDX')

#Create and add device
device = Device('Device')
nwb.add_device(device)

# Create an Imaging Plane
fs =1/(file1['time'][0][1]-file1['time'][0][0])
tt = file1['time'].ravel()
optical_channel = OpticalChannel(name='OpticalChannel',
                                 description='2P Optical Channel',
                                 emission_lambda=510.)
imaging_plane = nwb.create_imaging_plane(name='ImagingPlane',
                                         optical_channel=optical_channel,
                                         description='Imaging plane',
                                         device=device,
                                         excitation_lambda=488., 
                                         imaging_rate=fs,
                                         indicator='NLS-GCaMP6s',
                                         location='whole central brain')

#Dimensions
Xp = file1['dims'][0][0]
Yp = file1['dims'][0][1]
Zp = file1['dims'][0][2]
T = file1['dFF'].shape[1]
nCells = file1['dFF'].shape[0]

#Creates ophys ProcessingModule and add to file
ophys_module = ProcessingModule(name='ophys',
                                description='contains optical physiology processed data.')
nwb.add_processing_module(ophys_module)

**3.** Now transform the lists of indices into (xp,yp,zp) masks. With the masks created, we can add them to a plane segmentation class.

In [ ]:
def make_voxel_mask(indices, dims):
    """
    indices - List with voxels indices, e.g. [64371, 89300, 89301, ..., 3763753, 3763842, 3763843]
    dims - (height, width, depth) in pixels
    """
    voxel_mask = []
    for ind in indices:
        zp = np.floor(ind/(dims[0]*dims[1])).astype('int')
        rest = ind%(dims[0]*dims[1])
        yp = np.floor(rest/dims[0]).astype('int')
        xp = rest%dims[0]
        voxel_mask.append((xp,yp,zp,1))
    
    return voxel_mask

#Create Image Segmentation compartment
img_seg = ImageSegmentation()
ophys_module.add(img_seg)

#Create plane segmentation and add ROIs
ps = img_seg.create_plane_segmentation(description='plane segmentation',
                                       imaging_plane=imaging_plane)
#Call function
indices = file2['indices']
indptr = file2['indptr']
dims = np.squeeze(file1['dims'])

for start, stop in zip(indptr, indptr[1:]):
    voxel_mask = make_voxel_mask(indices[start:stop], dims)
    ps.add_roi(voxel_mask=voxel_mask)

**4.** Visualize voxel masks in 3D

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from itertools import cycle

%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for select, c in zip(range(len(indptr)-1),cycle(['r','g','k','b','m','w','y','brown'])):
    x, y, z, _ = np.array(ps['voxel_mask'][select]).T
    ax.scatter(x, y, z, c=c, marker='.')

**5.** With the ROIs created, we can add the dF/F data

In [ ]:
#DFF measures
dff = DfOverF(name='DfOverF')
ophys_module.add(dff)

#create ROI regions
roi_region = ps.create_roi_table_region(description='RoiTableRegion', 
                                        region=list(range(len(indptr)-1)))

#create ROI response series
dff_data = file1['dFF']
dFF_series = dff.create_roi_response_series(
    name='RoiResponseSeries', data=dff_data.T,
    unit='NA', rois=roi_region, timestamps=tt)

**6.** Adding a reference Volume Image

In [ ]:
#Creates GrayscaleVolume containers
grayscale_volume = GrayscaleVolume(name='GrayscaleVolume',
                                   data=file3['im'])
ophys_module.add(grayscale_volume)

**6.** Save trial times

In [ ]:
tt = file1['time'].ravel()
trialFlag = file1['trialFlag'].ravel()
trial_inds = np.hstack((0, np.where(np.diff(trialFlag))[0], trialFlag.shape[0]-1))
trial_times = tt[trial_inds]

for start, stop in zip(trial_times, trial_times[1:]):
    nwb.add_trial(start_time=start, stop_time=stop)

**7.** Save ball data

In [ ]:
behavior_mod = nwb.create_processing_module('behavior',
                             'holds processed behavior data')
behavior_mod.add(TimeSeries(name='ball_motion',
                            data=file1['ball'].ravel(),
                            timestamps=tt,
                            unit='unknown'))

**8.** Save body reference points positions over time

In [ ]:
#Re-arranges spatial data of body-points positions tracking
pos = file1['dlc']
nPoints = 8
pos_reshaped = pos.reshape((-1,nPoints,3))  #dims=(nSamples,nPoints,3)

# Creates a Position object
position = Position()

#Creates one SpatialSeries for each body-point position
for i in range(nPoints):
    position.create_spatial_series(name='SpatialSeries_'+str(i),
                                   data=pos_reshaped[:,i,:],
                                   timestamps=tt,
                                   reference_frame='Description defining what the zero-position is.')

behavior_mod.add(position)

**9.** Saving the NWB file

In [ ]:
#Saves to NWB file
path_to_files = ''
fname_nwb = 'file_1.nwb'
fpath_nwb = os.path.join(path_to_files, fname_nwb)
with NWBHDF5IO(fpath_nwb, mode='w') as io:
    io.write(nwb)
print('File saved with size: ', os.stat(fpath_nwb).st_size/1e6, ' mb')

**10.** Loading and checking the new NWB file

In [ ]:
#Loads NWB file
with NWBHDF5IO(fpath_nwb, mode='r') as io:
    nwb = io.read()
    print(nwb)